# Graphs

This notebook implements a generic graph that can be:
- Directed or undirected
- Weighted or unweighted

Representation:
- Adjacency list stored as a map of maps
- For unweighted graphs, edges use weight `1.0` by default

Included operations:
- Structure: add/remove vertices and edges, query neighbors/edges
- Traversals: BFS, DFS
- Shortest paths: Dijkstra (for non-negative weights)

We'll add a `Graph[T]` struct with a clean API and sample usage for both undirected/unweighted and directed/weighted graphs.

In [4]:
package main

import (
    "container/heap"
    "fmt"
    "math"
)

// SparseGraph represents an adjacency-list based graph
type SparseGraph[T comparable] struct {
    Directed bool
    Weighted bool
    adj      map[T]map[T]float64 // adjacency list: vertex -> neighbors -> weight
}

// NewSparseGraph creates a new sparse graph
func NewSparseGraph[T comparable](directed, weighted bool) *SparseGraph[T] {
    return &SparseGraph[T]{
        Directed: directed,
        Weighted: weighted,
        adj:      make(map[T]map[T]float64),
    }
}

// AddVertex adds a vertex if it doesn't exist
func (g *SparseGraph[T]) AddVertex(v T) {
    if _, exists := g.adj[v]; !exists {
        g.adj[v] = make(map[T]float64)
    }
}

// AddEdge adds an edge between two vertices
func (g *SparseGraph[T]) AddEdge(u, v T, weight float64) {
    g.AddVertex(u)
    g.AddVertex(v)
    
    if !g.Weighted {
        weight = 1.0
    }
    
    g.adj[u][v] = weight
    if !g.Directed {
        g.adj[v][u] = weight
    }
}

// HasEdge checks if an edge exists
func (g *SparseGraph[T]) HasEdge(u, v T) bool {
    if neighbors, exists := g.adj[u]; exists {
        _, hasEdge := neighbors[v]
        return hasEdge
    }
    return false
}

// GetWeight returns the weight of an edge
func (g *SparseGraph[T]) GetWeight(u, v T) (float64, bool) {
    if neighbors, exists := g.adj[u]; exists {
        if weight, hasEdge := neighbors[v]; hasEdge {
            return weight, true
        }
    }
    return 0, false
}

// Vertices returns all vertices
func (g *SparseGraph[T]) Vertices() []T {
    vertices := make([]T, 0, len(g.adj))
    for v := range g.adj {
        vertices = append(vertices, v)
    }
    return vertices
}

// Neighbors returns neighbors and their edge weights
func (g *SparseGraph[T]) Neighbors(v T) map[T]float64 {
    if neighbors, exists := g.adj[v]; exists {
        result := make(map[T]float64)
        for neighbor, weight := range neighbors {
            result[neighbor] = weight
        }
        return result
    }
    return make(map[T]float64)
}

// Edges returns all edges as (u, v, weight) tuples
func (g *SparseGraph[T]) Edges() []Edge[T] {
    var edges []Edge[T]
    seen := make(map[string]bool)
    
    for u, neighbors := range g.adj {
        for v, weight := range neighbors {
            if g.Directed {
                edges = append(edges, Edge[T]{From: u, To: v, Weight: weight})
            } else {
                // For undirected graphs, avoid duplicate edges
                key := fmt.Sprintf("%v-%v", u, v)
                reverseKey := fmt.Sprintf("%v-%v", v, u)
                if !seen[key] && !seen[reverseKey] {
                    edges = append(edges, Edge[T]{From: u, To: v, Weight: weight})
                    seen[key] = true
                }
            }
        }
    }
    return edges
}

// Edge represents an edge in the graph
type Edge[T comparable] struct {
    From, To T
    Weight   float64
}

// BFS performs breadth-first search
func (g *SparseGraph[T]) BFS(start T) []T {
    if _, exists := g.adj[start]; !exists {
        return []T{}
    }
    
    visited := make(map[T]bool)
    queue := []T{start}
    result := []T{}
    visited[start] = true
    
    for len(queue) > 0 {
        current := queue[0]
        queue = queue[1:]
        result = append(result, current)
        
        for neighbor := range g.adj[current] {
            if !visited[neighbor] {
                visited[neighbor] = true
                queue = append(queue, neighbor)
            }
        }
    }
    return result
}

// DFS performs depth-first search
func (g *SparseGraph[T]) DFS(start T) []T {
    if _, exists := g.adj[start]; !exists {
        return []T{}
    }
    
    visited := make(map[T]bool)
    stack := []T{start}
    result := []T{}
    
    for len(stack) > 0 {
        current := stack[len(stack)-1]
        stack = stack[:len(stack)-1]
        
        if visited[current] {
            continue
        }
        
        visited[current] = true
        result = append(result, current)
        
        // Add neighbors in reverse order for consistent traversal
        neighbors := make([]T, 0)
        for neighbor := range g.adj[current] {
            if !visited[neighbor] {
                neighbors = append(neighbors, neighbor)
            }
        }
        // Reverse neighbors
        for i := len(neighbors) - 1; i >= 0; i-- {
            stack = append(stack, neighbors[i])
        }
    }
    return result
}

// Priority queue implementation for Dijkstra
type PriorityItem[T comparable] struct {
    vertex   T
    distance float64
    index    int
}

type PriorityQueue[T comparable] []*PriorityItem[T]

func (pq PriorityQueue[T]) Len() int { return len(pq) }

func (pq PriorityQueue[T]) Less(i, j int) bool {
    return pq[i].distance < pq[j].distance
}

func (pq PriorityQueue[T]) Swap(i, j int) {
    pq[i], pq[j] = pq[j], pq[i]
    pq[i].index = i
    pq[j].index = j
}

func (pq *PriorityQueue[T]) Push(x interface{}) {
    n := len(*pq)
    item := x.(*PriorityItem[T])
    item.index = n
    *pq = append(*pq, item)
}

func (pq *PriorityQueue[T]) Pop() interface{} {
    old := *pq
    n := len(old)
    item := old[n-1]
    old[n-1] = nil
    item.index = -1
    *pq = old[0 : n-1]
    return item
}

// Dijkstra finds shortest paths from start vertex
func (g *SparseGraph[T]) Dijkstra(start T) map[T]float64 {
    if _, exists := g.adj[start]; !exists {
        return make(map[T]float64)
    }
    
    dist := make(map[T]float64)
    for v := range g.adj {
        dist[v] = math.Inf(1)
    }
    dist[start] = 0
    
    pq := make(PriorityQueue[T], 0)
    heap.Init(&pq)
    heap.Push(&pq, &PriorityItem[T]{vertex: start, distance: 0})
    
    for pq.Len() > 0 {
        item := heap.Pop(&pq).(*PriorityItem[T])
        u := item.vertex
        d := item.distance
        
        if d != dist[u] {
            continue // Already found better path
        }
        
        for v, weight := range g.adj[u] {
            if weight < 0 {
                panic("Dijkstra does not support negative edge weights")
            }
            
            newDist := d + weight
            if newDist < dist[v] {
                dist[v] = newDist
                heap.Push(&pq, &PriorityItem[T]{vertex: v, distance: newDist})
            }
        }
    }
    
    return dist
}

// String representation
func (g *SparseGraph[T]) String() string {
    kind := "Undirected"
    if g.Directed {
        kind = "Directed"
    }
    weight := "Unweighted"
    if g.Weighted {
        weight = "Weighted"
    }
    return fmt.Sprintf("SparseGraph %s %s |V|=%d |E|=%d", kind, weight, len(g.adj), len(g.Edges()))
}

## Examples

Below are quick examples showing undirected/unweighted and directed/weighted graph usage.

In [5]:
%%
// Undirected, unweighted graph usage
g := NewSparseGraph[string](false, false)
g.AddEdge("A", "B", 0)
g.AddEdge("A", "C", 0)
g.AddEdge("B", "D", 0)
g.AddEdge("C", "D", 0)

fmt.Println(g)
fmt.Println("Vertices:", g.Vertices())
fmt.Println("BFS from A:", g.BFS("A"))
fmt.Println("DFS from A:", g.DFS("A"))

SparseGraph Undirected Unweighted |V|=4 |E|=4
Vertices: [A B C D]
BFS from A: [A B C D]
DFS from A: [A C D B]


In [6]:
%%
// Directed, weighted graph + Dijkstra
gw := NewSparseGraph[string](true, true)
gw.AddEdge("A", "B", 4)
gw.AddEdge("A", "C", 2)
gw.AddEdge("C", "B", 1)
gw.AddEdge("B", "D", 5)
gw.AddEdge("C", "D", 8)

fmt.Println(gw)
fmt.Println("Dijkstra from A:", gw.Dijkstra("A"))

SparseGraph Directed Weighted |V|=4 |E|=5
Dijkstra from A: map[A:0 B:3 C:2 D:8]
Dijkstra from A: map[A:0 B:3 C:2 D:8]
